In [4]:
ctx = SparkSession.builder \
    .master("spark://h:7077") \
    .appName("bosch spark") \
    .getOrCreate()
file_dir = "/home/h/kaggle_fun/bosch/raw_data/"

In [7]:
numer_df = ctx.read.csv(file_dir + "train_numeric.csv", header=True, inferSchema=True)

In [8]:
ls /home/h/kaggle_fun/bosch/raw_data/train_numeric.csv

/home/h/kaggle_fun/bosch/raw_data/train_numeric.csv


In [ ]:
def load_csv_df(file_name):
    